In [1]:
from src.extract import extract
from src.export import export, export_dir, display_progress

from slippi import Game

from pymongo import MongoClient
from bson.binary import Binary
from scipy import sparse
import numpy as np
import pickle
import os

In [55]:
database_name = 'slippi'
collection_name = 'TCBL'

In [56]:
dir_full_path = os.path.join('/media/zack/vault/slippis/', collection_name)
print(f'Preparing to send contents of:\n\n - {dir_full_path}\n\nto [{database_name}.{collection_name}]')

Preparing to send contents of:

 - /media/zack/vault/slippis/TCBL

to [slippi.TCBL]


In [57]:
# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [ ]:
# export all .slp files in dir
export_dir(dir_full_path, database, collection_name)

[#...................] 27 of 375 - 7.2%%

In [ ]:
# number of documents in database
# each game produces two documens,
# one for each player
collection.count_documents({})

In [30]:
# retreive player data on single game from database
p1, p2 = tuple(collection.find({'game_id':'Game_20201119T171250.slp'}))[:2]

In [31]:
# convert istream from "sparse matrix bytestream" to numpy array
print(type(p1['istream']))
if isinstance(p1['istream'], bytes):
    p1['istream'] = pickle.loads( p1['istream']).toarray()
    p2['istream'] = pickle.loads( p2['istream']).toarray()
print(type(p1['istream']))

<class 'bytes'>
<class 'numpy.ndarray'>


In [32]:
print(f"{p1['name']} ({p1['code']}): {p1['character']}")
print(f"{p2['name']} ({p2['code']}): {p2['character']}")

NYTakeover (NYTA#893): FALCO
#Free Melee (TONG#898): CAPTAIN_FALCON


In [33]:
def presses(buttonstream):
    return int(np.sum(np.abs(np.diff(buttonstream, axis=0)))/2)

In [34]:
resultA = presses(p1['istream'][:, 9]) > presses(p2['istream'][:, 9])
resultB = presses(p1['istream'][:, 8]) > presses(p2['istream'][:, 8])
resultXY = presses(p1['istream'][:, [6,7]]) > presses(p2['istream'][:, [6,7]])
resultLR = presses(p1['istream'][:, [10,11]]) > presses(p2['istream'][:, [10,11]])
resultZ = presses(p1['istream'][:, 12]) > presses(p2['istream'][:, 12])

whoA = p1 if resultA else p2
whoB = p1 if resultB else p2
whoXY = p1 if resultXY else p2
whoLR = p1 if resultLR else p2
whoZ = p1 if resultZ else p2

print(f"{p1['name']}: {p1['character']}")
print('---VS---')
print(f"{p2['name']}: {p2['character']}")

print(f"\nMatch Length: {round(len(p1['istream'])/3600, 2)} minutes")
print(f"Match ID: {p1['game_id']}")

print(f"\nMost A presses: {presses(whoA['istream'][:, 9])}\n --- {whoA['name']} - {whoA['character']}")
print(f"\nMost B presses: {presses(whoB['istream'][:, 8])}\n --- {whoB['name']} - {whoB['character']}")
print(f"\nMost X or Y presses: {presses(whoXY['istream'][:, [6,7]])}\n --- {whoXY['name']} - {whoXY['character']}")
print(f"\nMost L or R presses: {presses(whoLR['istream'][:, [10,11]])}\n --- {whoLR['name']} - {whoLR['character']}")
print(f"\nMost Z presses: {presses(whoZ['istream'][:, 12])}\n --- {whoZ['name']} - {whoZ['character']}")

NYTakeover: FALCO
---VS---
#Free Melee: CAPTAIN_FALCON

Match Length: 2.26 minutes
Match ID: Game_20201119T171250.slp

Most A presses: 49
 --- NYTakeover - FALCO

Most B presses: 53
 --- NYTakeover - FALCO

Most X or Y presses: 147
 --- NYTakeover - FALCO

Most L or R presses: 115
 --- NYTakeover - FALCO

Most Z presses: 10
 --- #Free Melee - CAPTAIN_FALCON


In [35]:
if presses(p1['istream'][:, 6]) > presses(p1['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p1['name']} uses {jump_button} to jump")

if presses(p2['istream'][:, 6]) > presses(p2['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p2['name']} uses {jump_button} to jump")

NYTakeover uses Y to jump
#Free Melee uses Y to jump


In [36]:
collection.count_documents({'code' : 'GOOS#584'})

0

In [37]:
collection.count_documents({'code' : 'GHST#265'})

0

In [38]:
collection.count_documents({'code' : 'BENJ#230'})

0

In [39]:
collection.drop()